# Sparsifying Convolutional Neural Network using RVSM


In [3]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Training Parameters
learning_rate = 0.001
# learning_rate = 0.00005
num_steps = 500
batch_size = 128
display_step = 50

In [34]:
# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
# X = tf.zeros([num_input,1])
# Y = tf.zeros([1,num_classes])
keep_prob = tf.constant(0.2) # dropout (keep probability)

In [35]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [36]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)


In [37]:
# Define hard thresholding parameters
lamb = 0.0005
beta = 0.1
gamma = lamb/beta

# Define loss and optimizers
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

# AdamOptimizer

opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
grad = opt.compute_gradients(loss_op)
w = grad[2][1]
condition = tf.greater(tf.abs(w), np.sqrt(2*gamma))
u = tf.where(condition, w, tf.zeros_like(w))
new_grad = grad[2][0] + beta * (w - u)
grad[2] = (new_grad, w)
train_op1 = opt.apply_gradients(grad)

# Evaluate model

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [38]:
opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
opt.compute_gradients(loss_op)

[(<tf.Tensor 'gradients_1/Conv2D_grad/tuple/control_dependency_1:0' shape=(5, 5, 1, 32) dtype=float32>,
  <tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/Conv2D_1_grad/tuple/control_dependency_1:0' shape=(5, 5, 32, 64) dtype=float32>,
  <tf.Variable 'Variable_1:0' shape=(5, 5, 32, 64) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/MatMul_grad/tuple/control_dependency_1:0' shape=(3136, 1024) dtype=float32>,
  <tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/MatMul_1_grad/tuple/control_dependency_1:0' shape=(1024, 10) dtype=float32>,
  <tf.Variable 'Variable_3:0' shape=(1024, 10) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/BiasAdd_grad/tuple/control_dependency_1:0' shape=(32,) dtype=float32>,
  <tf.Variable 'Variable_4:0' shape=(32,) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/BiasAdd_1_grad/tuple/control_dependency_1:0' shape=(64,) dtype=float32>,
  <tf.Variable 'Variable_5:0' shape=(64,) dty

In [39]:
# Initialize save function
saver = tf.train.Saver()

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [97]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op1, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))
    wbar = tf.Variable(w.eval())
    saver.save(sess, 'mnist1.chkp')


Step 1, Minibatch Loss= 47763.3359, Training Accuracy= 0.164
Step 50, Minibatch Loss= 3496.3965, Training Accuracy= 0.781
Step 100, Minibatch Loss= 3022.1284, Training Accuracy= 0.828
Step 150, Minibatch Loss= 668.8149, Training Accuracy= 0.953
Step 200, Minibatch Loss= 759.8076, Training Accuracy= 0.914
Step 250, Minibatch Loss= 290.3637, Training Accuracy= 0.969
Step 300, Minibatch Loss= 586.8425, Training Accuracy= 0.938
Step 350, Minibatch Loss= 353.7922, Training Accuracy= 0.961
Step 400, Minibatch Loss= 199.9929, Training Accuracy= 0.977
Step 450, Minibatch Loss= 65.3822, Training Accuracy= 0.984
Step 500, Minibatch Loss= 819.1185, Training Accuracy= 0.938
Optimization Finished!
Testing Accuracy: 0.98046875


In [124]:
# Evaluate with u

condition = tf.greater(tf.abs(wbar), np.sqrt(2*gamma))
ubar = tf.where(condition, w, tf.zeros_like(w))
weights['wd1'] = ubar
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy_u = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Evaluate with not u

condition = tf.greater(tf.abs(wbar), np.sqrt(2*gamma))
unot = tf.where(condition, tf.zeros_like(w), w)
weights['wd1'] = unot
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy_unot = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


init = tf.global_variables_initializer()

In [125]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, 'mnist1.chkp')
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    
    # Evaluate with w
    acc = sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0})
    print('Accuracy with w: {:.8}'.format(acc))
    
    # Evaluate with u        
    acc = sess.run(accuracy_u, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0})
    print('Accuracy with u: {:.8}'.format(acc))
    
    # Evaluate with unot     
    acc = sess.run(accuracy_unot, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0})
    print('Accuracy with u not: {:.8}'.format(acc))  


INFO:tensorflow:Restoring parameters from mnist1.chkp
Accuracy with w: 0.9804687500
Accuracy with u: 0.9804687500
Accuracy with u not: 0.1289062500


In [31]:
tf.reset_default_graph()

In [106]:
a = tf.random_normal([3,3])
condition = tf.greater(tf.abs(a), np.sqrt(2*gamma))
b = tf.where(condition, a, tf.zeros_like(a))

init = tf.global_variables_initializer()


In [123]:
with tf.Session() as sess:
    sess.run(init)
    print((tf.reduce_sum(tf.cast(tf.abs(wbar.eval())>0.1,tf.float32))).eval())
    

2944295.0


In [121]:
nonzeros = 266969
zeros = 2944295

In [130]:
tf.trainable_variables().remove()

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(5, 5, 32, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_4:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'Variable_5:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'Variable_8:0' shape=(3, 3) dtype=float32_ref>,
 <tf.Variable 'Variable_9:0' shape=(3, 3) dtype=float32_ref>,
 <tf.Variable 'Variable_10:0' shape=(3, 3) dtype=float32_ref>,
 <tf.Variable 'Variable_11:0' shape=(3136, 1024) dtype=float32_ref>]